In [ ]:
# import library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import spearmanr

In [ ]:
# setting
plt.rcParams['font.family']= 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Arial']
plt.rcParams['font.size'] = 18
plt.rcParams["figure.dpi"] = 200

In [ ]:
# Data
file_path = "../data/ChIP-seq/ChIP_H3K4me2_H2A.Z_WT_arp6_pie1.rpkm.tsv"
df_chip = pd.read_csv(file_path, sep="\t", index_col=0)
swr1_target = pd.read_csv("../data/list_of_target_genes/SWR1-C-target(n=6096).bed",sep="\t",header=None).iloc[:,3]
df_chip

In [ ]:
# difference
hm_list = ["H3","H3K4me2","H2A.Z"]
gt_list = ["arp6","pie1"]
dict_tmp = {}
for hm in hm_list:
    for gt in gt_list:
        dict_tmp[f"Δ{hm}_{gt}"] = df_chip[f"{hm}_{gt}_rep1"] - df_chip[f"{hm}_WT_rep1"]

df_diff = pd.DataFrame(dict_tmp)
df_diff

In [ ]:
# Supplementary Fig. 5b
fig,ax = plt.subplots(1,1,figsize=(4,4))
hm = "H2A.Z"
sns.scatterplot(data=df_diff,
                x=f"Δ{hm}_arp6",
                y=f"Δ{hm}_pie1",
                s=2,alpha=0.5,c="tab:grey")
x = df_diff[f"Δ{hm}_arp6"]
y = df_diff[f"Δ{hm}_pie1"]
ax.set_xlim(np.percentile(x,0.1),np.percentile(x,99.9))
ax.set_ylim(np.percentile(x,0.1),np.percentile(x,99.9))
ax.set_aspect('equal', adjustable='box')
ax.set_title(f"ρ={spearmanr(x,y)[0]:.3f}")

In [ ]:
# Supplementary Fig. 5d
hm_list = ["H2A.Z","H3K4me2"]
mask = df_chip.index.isin(swr1_target)

fig,ax = plt.subplots(1,2,figsize=(10,4))

for i in range(2):
    hm = hm_list[i]
    print(hm)

    # plot
    sns.scatterplot(data=df_chip[~mask],
                    x=f"{hm}_WT_rep1",
                    y=f"{hm}_arp6_rep1",
                    s=2,alpha=0.5,ax=ax[i],c="tab:grey")
    sns.scatterplot(data=df_chip[mask],
                    x=f"{hm}_WT_rep1",
                    y=f"{hm}_arp6_rep1",
                    s=2,alpha=0.5,ax=ax[i],c="blue")

    # format figure
    x = df_chip[f"{hm}_WT_rep1"]
    ax[i].set_xlim(0,np.percentile(x,99))
    ax[i].set_ylim(0,np.percentile(x,99))
    ax[i].set_aspect('equal', adjustable='box')
    ax[i].set_xlabel(f"{hm} in WT")
    ax[i].set_ylabel(f"{hm} in arp6")

In [ ]:
# Supplementary Fig. 5e
gt_list = ["arp6","pie1"]

fig,ax = plt.subplots(1,2,figsize=(10,4))

for i in range(2):
    gt = gt_list[i]

    # plot
    sns.scatterplot(data=df_diff,
                    x=f"ΔH2A.Z_{gt}",
                    y=f"ΔH3K4me2_{gt}",
                    s=2,alpha=0.5,ax=ax[i],c="tab:grey")

    # format figure
    x = df_diff[f"ΔH2A.Z_{gt}"]
    y = df_diff[f"ΔH3K4me2_{gt}"]
    ax[i].set_xlim(np.percentile(x,0.1),np.percentile(x,99.9))
    ax[i].set_ylim(np.percentile(y,0.1),np.percentile(y,99.9))
    ax[i].set_box_aspect(1)
    ax[i].set_xlabel(f"H2A.Z change in {gt}")
    ax[i].set_ylabel(f"H3K4me2 change in {gt}")

    # spearman's R
    r, _ = spearmanr(x,y)
    print(f"{gt} : {r:.2e}")